Runs two basic convolutional networks on the pilot dataset.

### Things to do next
- Figure out how to do regression (see below).
- ~~Figure out how to do multitask learning (i.e. try to predict the different reps and different genes).~~
- ~~Run on the scale up dataset.~~
- Read up on the "Interpreting a DragoNN model using filter visualization" and "Interpreting data with a DragoNN model" in the Dragonn tutorial.

### Installing Dragonn
- Clone from https://github.com/kundajelab/dragonn
- ```python setup.py```
    - I needed to ```brew install geos```

In [1]:
from dragonn import models
from dragonn.plot import add_letters_to_axis

from sklearn.model_selection import train_test_split

from collections import OrderedDict
from pprint import pprint
from warnings import warn

import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

Using Theano backend.
Using gpu device 0: Tesla K80 (CNMeM is disabled, cuDNN 5005)
/usr/local/anaconda3/envs/regression_dragonn_new/lib/python2.7/site-packages/theano/tensor/signal/downsample.py:6: UserWarning: downsample module has been moved to the theano.tensor.signal.pool module.
  "downsample module has been moved to the theano.tensor.signal.pool module.")


Read in the data.
- Samples are sequences that will be one hot encoded
- Try and predict the normalized values. **I couldn't figure out how to do regression with Dragonn, so I just rounded the values to 0 or 1 based on the median.**

In [2]:
key_to_seq = OrderedDict()

with open("../data/Scaleup_counts_sequences/ScaleUpDesign1.sequences.txt") as f:
    for line in f:
        key, seq = line.strip().split()
        
        # TODO: Figure out if this is an OK thing to do. 'N' basically means the 
        # sequencing software couldn't figure out what the base was...?
        if "N" in seq:
            warn("Replacing 'N' bases in seq with 'A' in seq {}.".format(seq))
            seq = seq.replace("N", "A")
        
        assert key not in key_to_seq
        key_to_seq[key] = seq
        
with open("../data/Scaleup_counts_sequences/ScaleUpDesign2.sequences.txt") as f:
    for line in f:
        key, seq = line.strip().split()
        
        if "N" in seq:
            warn("Replacing 'N' bases in seq with 'A' in seq {}.".format(seq))
            seq = seq.replace("N", "A")
        
        assert key not in key_to_seq
        key_to_seq[key] = seq
        
pprint(key_to_seq.items()[:5])

print "{} total sequences of length {}".format(len(key_to_seq), len(key_to_seq.values()[0]))

/usr/local/anaconda3/envs/regression_dragonn_new/lib/python2.7/site-packages/ipykernel/__main__.py:21: UserWarning: Replacing 'N' bases in seq with 'A' in seq NNNNNNNNNGATCCCGCGGCCGTGTTTTCCTGGTGGCCCGGCCGTGCCTGAGGTTTCTCCCCGAGCCGCCGCCTCTGCGGGCTCCCGGGTGCCCTTGCCCTCACGGTCCCCGGCCCTCGCCCGTCTGTGCCCTCTTCCCCGCCCG.
/usr/local/anaconda3/envs/regression_dragonn_new/lib/python2.7/site-packages/ipykernel/__main__.py:21: UserWarning: Replacing 'N' bases in seq with 'A' in seq NNNNGATCCCGCGGCCGTGTTTTCCTGGTGGCCCGGCCGTGCCTGAGGTTTCTCCCCGAGCCGCCGCCTCTGCGGGCTCCCGGGTGCCCTTGCCCTCACGGTCCCCGGCCCTCGCCCGTCTGTGCCCTCTTCCCCGCCCGCCGCC.
/usr/local/anaconda3/envs/regression_dragonn_new/lib/python2.7/site-packages/ipykernel/__main__.py:21: UserWarning: Replacing 'N' bases in seq with 'A' in seq NNNNNNNNNNNNNNNNNNGAATTCAATGGAGTGGACTGGAGTGCTGTGGGGTGGAGTGGAATGGAGTGTAGTTGAATGGAGTGGAATGGAATGCGATGGAATGGAGTGGAGTTGAGCAGAGTGAAGTGGAAAGGTGTAGAATGGA.
/usr/local/anaconda3/envs/regression_dragonn_new/lib/python2.7/site-packages/ipyke

[('H1hesc_1_0_0_chr20_30310735',
  'GGGAGCCCAGAAGGCGACACAGGAATTGCGAAGCTCAGGAACCAGCCCCCTCGCTTGCTTCCTCCTCCATCGCCCGGATCGAGGGCGGCCGCTCCGCAGCCGCGGCCTCCTGCCACCCGGGAGCCCAGCCCCCTCTCTCTTGCAC'),
 ('H1hesc_1_0_1_chr20_30310735',
  'CCCAGAAGGCGACACAGGAATTGCGAAGCTCAGGAACCAGCCCCCTCGCTTGCTTCCTCCTCCATCGCCCGGATCGAGGGCGGCCGCTCCGCAGCCGCGGCCTCCTGCCACCCGGGAGCCCAGCCCCCTCTCTCTTGCACGCCCC'),
 ('H1hesc_1_0_2_chr20_30310735',
  'AAGGCGACACAGGAATTGCGAAGCTCAGGAACCAGCCCCCTCGCTTGCTTCCTCCTCCATCGCCCGGATCGAGGGCGGCCGCTCCGCAGCCGCGGCCTCCTGCCACCCGGGAGCCCAGCCCCCTCTCTCTTGCACGCCCCTTGGC'),
 ('H1hesc_1_0_3_chr20_30310735',
  'GACACAGGAATTGCGAAGCTCAGGAACCAGCCCCCTCGCTTGCTTCCTCCTCCATCGCCCGGATCGAGGGCGGCCGCTCCGCAGCCGCGGCCTCCTGCCACCCGGGAGCCCAGCCCCCTCTCTCTTGCACGCCCCTTGGCTCTCC'),
 ('H1hesc_1_0_4_chr20_30310735',
  'AGGAATTGCGAAGCTCAGGAACCAGCCCCCTCGCTTGCTTCCTCCTCCATCGCCCGGATCGAGGGCGGCCGCTCCGCAGCCGCGGCCTCCTGCCACCCGGGAGCCCAGCCCCCTCTCTCTTGCACGCCCCTTGGCTCTCCGCCTC')]
487137 total sequences of length 145


In [3]:
data = {}
cell_types =  ["HepG2", "K562"]
promoters = ["SV40P", "minP"]
design_names = ["ScaleUpDesign1", "ScaleUpDesign2"]

for cell_type in cell_types:
    for promoter in promoters:
        experiment_key = (cell_type, promoter)
        data[experiment_key] = {}

        for design_name in design_names:

            with open("../data/Scaleup_normalized/{}_{}_{}_mRNA_Rep1.normalized".format(cell_type, design_name, promoter)) as f:
                for line in f:
                    parts = line.strip().split()

                    key = parts[0]
                    val = float(parts[1])
                    if parts[2] == "1":
                        data[experiment_key][key] = val

            with open("../data/Scaleup_normalized/{}_{}_{}_mRNA_Rep2.normalized".format(cell_type, design_name, promoter)) as f:
                for line in f:
                    parts = line.strip().split()

                    key = parts[0]
                    val = float(parts[1])
                    if parts[2] == "1" and key in data[experiment_key]:
                        data[experiment_key][key] = (val + data[experiment_key][key]) / 2.0
            
print "Data from experiment {}:".format(data.items()[0][0])
pprint(data.items()[0][1].items()[:5])

Data from experiment ('HepG2', 'minP'):
[('Huvec_15_5_12_chr9_134025315', -0.8987649255400356),
 ('K562_7_2_10_chr5_77817535', -2.6900887233895894),
 ('K562_8_39_18_chr11_5602495', -0.9276572495402888),
 ('Huvec_5_556_25_chr16_3727095', -0.6722025396114724),
 ('Hepg2_8_141_16_chr7_514575', 1.8181443314468417)]


In [4]:
for key, val in data.items():
    print "Experiment {} has {} measurements.".format(key, len(val))

Experiment ('HepG2', 'minP') has 332929 measurements.
Experiment ('K562', 'minP') has 332929 measurements.
Experiment ('HepG2', 'SV40P') has 296463 measurements.
Experiment ('K562', 'SV40P') has 296463 measurements.


In [5]:
# One hot encode DNA sequences the standard way.
bases = ['A', 'T', 'C', 'G']

def one_hot_encode_seq(seq):
    result = np.zeros((len(bases), len(seq)))
    
    for i, base in enumerate(seq):
        result[bases.index(base), i] = 1

    return result

def seqs_to_encoded_matrix(seqs):
    # Wrangle the data into a shape that Dragonn wants.
    result = np.concatenate(
        map(one_hot_encode_seq, seqs)
    ).reshape(
        len(seqs), 1, len(bases), len(seqs[0])
    )
    
    # Check we actually did the encoding right.
    for i in range(len(seqs)):
        for j in range(len(seqs[0])):
            assert sum(result[i, 0, :, j]) == 1
    
    return result

- We formulate this as a multi-task learning problem, where each cell type and promoter combo is a task, i.e. the tasks are 
- Some of the normalized scores are too noisy (as determined by the SHARPR software). For now, we only consider a sequence if it has a good measurement for all of the tasks
  

In [6]:
valid_keys = list(reduce(
    lambda acc, d: acc.intersection(d.keys()), 
    data.values()[1:], 
    set(data.values()[0].keys())
))

print "{} sequences have measurements for all tasks.".format(len(valid_keys))

277409 sequences have measurements for all tasks.


In [7]:
X = seqs_to_encoded_matrix([key_to_seq[key] for key in valid_keys])

In [8]:
X.shape

(277409, 1, 4, 145)

In [9]:
# Just round to the median, to make this a classification task for now.
experiment_labels = []
for experiment_key, key_to_normalized in data.items():
    
    filtered_normalized = [key_to_normalized[key] for key in valid_keys]
    
    median = np.median(filtered_normalized)
    experiment_labels.append(
        np.array(map(lambda val: val > median, filtered_normalized)).reshape(-1, 1)
    )

y = np.hstack(experiment_labels)

In [10]:
y.shape

(277409, 4)

In [11]:
X[:5, :, :, :]

array([[[[ 0.,  0.,  0., ...,  0.,  0.,  0.],
         [ 1.,  1.,  1., ...,  1.,  0.,  0.],
         [ 0.,  0.,  0., ...,  0.,  0.,  0.],
         [ 0.,  0.,  0., ...,  0.,  1.,  1.]]],


       [[[ 0.,  1.,  1., ...,  0.,  0.,  0.],
         [ 0.,  0.,  0., ...,  1.,  0.,  0.],
         [ 1.,  0.,  0., ...,  0.,  1.,  1.],
         [ 0.,  0.,  0., ...,  0.,  0.,  0.]]],


       [[[ 0.,  1.,  0., ...,  0.,  0.,  0.],
         [ 0.,  0.,  1., ...,  0.,  1.,  0.],
         [ 1.,  0.,  0., ...,  0.,  0.,  1.],
         [ 0.,  0.,  0., ...,  1.,  0.,  0.]]],


       [[[ 1.,  1.,  1., ...,  1.,  0.,  1.],
         [ 0.,  0.,  0., ...,  0.,  0.,  0.],
         [ 0.,  0.,  0., ...,  0.,  1.,  0.],
         [ 0.,  0.,  0., ...,  0.,  0.,  0.]]],


       [[[ 0.,  1.,  0., ...,  0.,  0.,  0.],
         [ 1.,  0.,  0., ...,  1.,  1.,  0.],
         [ 0.,  0.,  0., ...,  0.,  0.,  0.],
         [ 0.,  0.,  1., ...,  0.,  0.,  1.]]]])

In [12]:
y[:5, :]

array([[ True,  True, False,  True],
       [False, False, False,  True],
       [ True,  True,  True,  True],
       [ True, False, False, False],
       [ True,  True,  True,  True]], dtype=bool)

In [13]:
X_train, X_valid, y_train, y_valid = train_test_split(
    X, y, test_size=0.2, random_state=42
)

Start with a very small model. Train and plot the train and validation loss.

In [14]:
model = models.SequenceDNN_Regression(
    seq_length=X_train.shape[3],
    num_filters=[100, 100, 100],
    conv_width=[15, 15, 15],
    pool_width=40,
    num_tasks=y_train.shape[1]
)

In [15]:
model.train(X_train, y_train, (X_valid, y_valid))

Training model (* indicates new best result)...
Epoch 1:
Train Task 0: Mean Squared Error: 0.2472	Mean Absolute Error: 0.4946	 Median Absolute Error: 0.4936	 R2 Score: 0.0110
Task 1: Mean Squared Error: 0.2477	Mean Absolute Error: 0.4955	 Median Absolute Error: 0.4959	 R2 Score: 0.0092
Task 2: Mean Squared Error: 0.2478	Mean Absolute Error: 0.4953	 Median Absolute Error: 0.4956	 R2 Score: 0.0090
Task 3: Mean Squared Error: 0.2471	Mean Absolute Error: 0.4948	 Median Absolute Error: 0.4952	 R2 Score: 0.0114
Valid Task 0: Mean Squared Error: 0.2475	Mean Absolute Error: 0.4948	 Median Absolute Error: 0.4939	 R2 Score: 0.0099
Task 1: Mean Squared Error: 0.2484	Mean Absolute Error: 0.4962	 Median Absolute Error: 0.4964	 R2 Score: 0.0064
Task 2: Mean Squared Error: 0.2483	Mean Absolute Error: 0.4958	 Median Absolute Error: 0.4961	 R2 Score: 0.0067
Task 3: Mean Squared Error: 0.2476	Mean Absolute Error: 0.4953	 Median Absolute Error: 0.4956	 R2 Score: 0.0095 *
Epoch 2:
Train Task 0: Mean Squar

In [17]:
fn = "model"
models.SequenceDNN_Regression.save(model, fn)

In [18]:
def print_loss(model):
    train_losses, valid_losses = [np.array([epoch_metrics['Loss'] for epoch_metrics in metrics])
                                  for metrics in (model.train_metrics, model.valid_metrics)]

    # Pretty sure early stopping works by taking the mean of losses, might want to double check
    train_losses = train_losses.mean(axis=1)
    valid_losses = valid_losses.mean(axis=1)

    f = plt.figure(figsize=(10, 4))
    ax = f.add_subplot(1, 1, 1)
    
    ax.plot(range(len(train_losses)), train_losses, label='Training',lw=4)
    ax.plot(range(len(train_losses)), valid_losses, label='Validation', lw=4)
    
    min_loss_indx = min(enumerate(valid_losses), key=lambda x: x[1])[0]
    ax.plot([min_loss_indx, min_loss_indx], [0, 1.0], 'k--', label='Early Stop')
    ax.legend(loc="upper right")
    ax.set_ylabel("Loss")
    ax.set_ylim((0.0,1.0))
    ax.set_xlabel("Epoch")
    plt.show()

In [19]:
print_loss(model)

KeyError: 'Loss'

In [ ]:
def plot_sequence_filters(dnn):
    fig = plt.figure(figsize=(15, 8))
    fig.subplots_adjust(hspace=0.1, wspace=0.1)
    conv_filters = dnn.get_sequence_filters()
    num_plots_per_axis = int(len(conv_filters)**0.5) + 1
    for i, conv_filter in enumerate(conv_filters):
        ax = fig.add_subplot(num_plots_per_axis, num_plots_per_axis, i+1)
        add_letters_to_axis(ax, conv_filter.T)
        ax.axis("off")
        ax.set_title("Filter %s" % (str(i+1)))

In [ ]:
plot_sequence_filters(model)

Try a larger model.

In [ ]:
multi_filter_model = models.SequenceDNN(
    seq_length=X_train.shape[3],
    num_filters=[15, 15, 15],
    conv_width=[15, 15, 15],
    num_tasks=y_train.shape[1],
    dropout=0.1
)

In [ ]:
multi_filter_model.train(X_train, y_train, (X_valid, y_valid))

In [ ]:
print_loss(multi_filter_model)